In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
address = '0x3e5126c8dd68db20246e51bc31cb087a28003d2e'

"0x3e5126c8dd68db20246e51bc31cb087a28003d2e"

In [ ]:
def address_graph_expand(addr,exclude)
  addr_list = addr.map {|x| x.to_a[0][1]}.uniq

  expend_all_addr = Transfer.where(from:addr_list).map {|x| x.to} + Transfer.where(to:addr_list).map {|x| x.from} + addr_list
  expend_addr = expend_all_addr.uniq.filter { |x| not exclude[x]}
  ret = expend_addr.map {|x| [["(input)",x]].to_h }
      
  return ret
end

def to_graph(addr,min_amount=0, max_connect=100)
  addr_list = addr.map {|x| x.to_a[0][1]}.uniq

  exclude_addr = addr_list.map { |addr|
      [addr, Transfer.where(from:addr).count, Transfer.where(to:addr).count, Transfer.where(from:addr).sum(:amount)/1e18, Transfer.where(to:addr).sum(:amount)/1e18]
  }.filter{ |x| x[2]>max_connect and x[1]>max_connect}

  addr_list = addr_list.filter {|x| not exclude_addr.map{|x| x[0]}.include?(x) }

  graph = "digraph {\n" +  
    exclude_addr.map {|addr| 
      x=Address.find_by_addr(addr[0]);  
      %Q( "#{x.tag ? x.tag + ' @'+x.addr[-4,4] : x.addr }" [shape=doubleoctagon style=filled] ) + ";\n" + 
      %Q( "group @#{x.addr[-4,4]} trans #{addr[1]+addr[2]}"[style=filled] ) + ";\n" +
      %Q( "#{x.tag ? x.tag + ' @'+x.addr[-4,4] : x.addr }" -> "group @#{x.addr[-4,4]} trans #{addr[1]+addr[2]}" [label=#{addr[3].round(2)}] ) + ";\n" +
      %Q( "group @#{x.addr[-4,4]} trans #{addr[1]+addr[2]}" -> "#{x.tag ? x.tag + ' @'+x.addr[-4,4] : x.addr }" [label=#{addr[4].round(2)}] ) + ";\n"
    }.join("") + 
    Transfer.where(from:addr_list).where(to:addr_list).where("amount > ?",min_amount * 1e18).map {|x|
      %Q( "#{x.ar_from.tag ? x.ar_from.tag + ' @'+x.from[-4,4] : x.from }" -> "#{ x.ar_to.tag ? x.ar_to.tag + ' @'+x.to[-4,4] : x.to}" [label=#{(x.amount/1e18).round(2)}] )
    }.join(";\n") + "\n}"
  url = "https://quickchart.io/graphviz?graph=#{graph.gsub(/\n/,"")}"
  body = Faraday.get(url).body
end



:to_graph

In [ ]:
exclude = {}
exclude.merge! Address.where("tag LIKE ?","%Hot Wallet%").select(:addr).map {|x| [x.addr,true]}.to_h
exclude.merge! Address.where("tag LIKE ?","%Bridge%").select(:addr).map {|x| [x.addr,true]}.to_h
exclude.merge! Address.where(is_contract:true).select(:addr).map {|x| [x.addr,true]}.to_h

level =  []
level << [{address=>address}]


NameError: uninitialized constant Address

In [ ]:
puts saved_count = level.last.count 
level << address_graph_expand(level.last,exclude)


12476


Interrupt: 

In [ ]:
def auto_tag(addr,name)
    addr_list = addr.map {|x| x.to_a[0][1]}.uniq

    bet_transfer_ratio = Address.where(addr:addr_list).map {|x|
        call_method_count = Tx.where('"to"=? or "from"=?',x.addr,x.addr).where("method_name=? or method_name=?","betBear","betBull").count; 
        transfer_count = Transfer.where('"to"=? or "from"=?',x.addr,x.addr).where(method_name:"Transfer").count; 
        [x.addr, call_method_count ,transfer_count ]
    }
    .map {|x| [x[0],x[1].to_f/x[2],x[1],x[2]]}
    .sort {|x,y| x[1]<=>y[1]}

    panbot_account_list = bet_transfer_ratio.filter {|x| not( x[1]<=0.05 ) }.map {|x| x[0]}
    money_account_list = bet_transfer_ratio.filter {|x| x[1]<=0.05  }.map {|x| x[0]}

    puts "panbot_account_list.count = #{panbot_account_list.count}"
    puts "money_account_list.count = #{money_account_list.count}"

    panbot_account_list.each_with_index {|x,i| addr=Address.find_by_addr(x);addr.tag="#{name} bot #{i}"; addr.save}
    money_account_list.each_with_index {|x,i| addr=Address.find_by_addr(x);addr.tag="#{name} money #{i}"; addr.save}
    
end

auto_tag(level.last,"tiger")


panbot_account_list.count = 2
money_account_list.count = 2


["0x9572a3bf6f57e6d022eada8119be4dd3c6b22421", "0xdb76f4fb6734d4db2eac42b7bc7955a96a274661"]

In [ ]:
IRuby.html to_graph(level[1],0)

Interrupt: 

In [ ]:
#   expend_all_addr = Transfer.where(from:addr_list).map {|x| x.to} + Transfer.where(to:addr_list).map {|x| x.from} + addr_list
#     expend_addr = expend_all_addr.uniq.filter { |x| not exclude[x]}
#     expend_addr

# from = Transfer.where(method_name:"Transfer").group(:from).count
# from.sort {|x,y| -1*(x[1]<=>y[1])}[0,100].map { |x|
#   addr = Address.find_by_addr(x[0])
#   [addr.tag, x[0], x[1]]
# }


# addr_list = level[4].map {|x| x.to_a[0][1]}.uniq
# from_count = Transfer.where(from:addr_list).group(:from).count
# to_count = Transfer.where(to:addr_list).group(:to).count
# trans_count = from_count.merge(to_count){|key, oldval, newval| newval + oldval}
# to_count.to_a.sort {|x,y| -1*(x[1]<=>y[1])}

["..."]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>